## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
!pip install sentencepiece

In [3]:
!pip install pypdf

In [4]:
from pypdf import PdfReader

# creating a pdf reader object
reader = PdfReader('Tõde_ja_õigus_I.pdf')

# printing number of pages in pdf file
print(len(reader.pages))

text=''

for page in reader.pages:
  # extracting text from page
  txt_ = page.extract_text()
  text = f'{text} {txt_}'

865


In [5]:
text[:200]

' 1Tõde ja õigus I\nA. H. T ammsaar e\nExported from Wikisource on 14. märts 2024 2 \n \n \n A. H. T AMMSAARE\n \nTÕDE JA ÕIGUS\nI\n \n \nKIRJASTUS „EESTI RAAMA T“ · T ALLINN 1974\n \n\xa0I\xa0 · \xa0II\xa0 · III · IV · V · VI'

In [6]:
# read it in to inspect it
# with open('input.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

In [7]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1026340


In [8]:
# let's look at the first 1000 characters
print(text[:1000])

 1Tõde ja õigus I
A. H. T ammsaar e
Exported from Wikisource on 14. märts 2024 2 
 
 
 A. H. T AMMSAARE
 
TÕDE JA ÕIGUS
I
 
 
KIRJASTUS „EESTI RAAMA T“ · T ALLINN 1974
 
 I  ·  II  · III · IV · V · VI · VII · VIII · IX · X 
XI · XII · XIII · XIV · XV · XVI · XVII  · XVIII  ·
XIX · XX 
XXI · XXII  · XXIII  · XXIV  · XXV  · XXVI  · XXVII  ·
XXVIII  · XXIX  · XXX  
XXXI  · XXXII  · XXXIII  · XXXIV  · XXXV  · XXXVI
· XXXVII  · XXXVIII  · XXXIX 3 I
See oli läinud aastasaja kolmanda veerandi lõpul. Päike
lähenes silmapiirile, seistes sedavõrd madalas, et enam ei
ulatunud valgustama ei mäkke ronivat hobust, kes
puutelgedega vankrit vedas, ei vankril istuvat noort naist
ega ka ligi kolmekümnelist meest, kes kõndis vankri kõrval.
Varsti jõudsid teelised mäerinnakul nii kõr gele, et päikeses
helendama lõid mehe nägu – laiavõitu, tugevate
lõuapäradega, terassilmadega, lühikese, kuid tiheda musta
habemega –, naise nukrad silmad, look ja hobuse kikkis
kõrvadega pea.
„Seal ta ongi, see V argamäe,“ l

In [9]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !%'(),-./0123456789:;?ABCDEFGHIJKLMNOPRSTUVWX[]_abcdefghijklmnoprstuvwxy ·ÄÕÖÜäõöüš–’“„…↑✻
92


In [10]:
import sentencepiece as spm

In [11]:
with open('text.txt', 'w') as f:
  f.write(text)

In [12]:
vocab_size=1000
spm.SentencePieceTrainer.train(input="text.txt", model_prefix='m', vocab_size=vocab_size)

In [13]:
!ls

m.model  m.vocab  sample_data  text.txt  Tõde_ja_õigus_I.pdf


In [14]:
sp = spm.SentencePieceProcessor(model_file='m.model')

In [15]:
sp.encode('tere')

[179, 128]

In [16]:
sp.decode([179, 128])

'tere'

In [17]:
# # create a mapping from characters to integers
# stoi = { ch:i for i,ch in enumerate(chars) }
# itos = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

#use sentencepiece tokenizer
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: sp.encode(s) # encoder: take a string, output a list of integers
decode = lambda l: sp.decode(l) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[115, 12, 12, 4, 10, 135, 128]
hii there


In [18]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([350143]) torch.int64
tensor([480, 239,  52,  78,   9, 401,   4, 223, 265,   5, 509,   5,  79,  87,
         33,  33,  81, 149,   4,   8, 282,   0,  82, 320,  64,  11,   4,   0,
         36,  32,  33,   4,   0, 302,  54,  32,  16,  36,   0,   8,  42, 480,
        229,   5, 529,  10,   6,   4, 233, 378, 233, 229,   4, 233, 265,   5,
        509,   5,  79, 265, 503, 503, 514, 471, 471, 620, 686,  79,   0, 686,
          4, 304, 471,   4,   0, 223,   0, 514,   4, 223, 217, 223, 620, 304,
        471, 514, 239,   0, 514,  15, 686, 686, 514, 239, 223,   4, 620, 471,
        471, 503, 471,  79, 432,   4,   0,  79, 265, 627, 627, 223, 409, 409,
        480, 380, 247, 229,   4, 223,   4,   0,   4, 223, 223,   4,   0,   4,
        223, 223, 223,   4,   0,   4, 223, 435,   4,   0,  67,   4,   0,  67,
        223,   4,   0,  67, 223, 223,   4,   0,  67, 223, 223, 223,   4,   0,
          4, 223, 996,   4,   0,   4, 996,   4, 996, 223,   4,   0,   4, 996,
        223, 223,   4,   0,   4

In [19]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [20]:
block_size = 8
train_data[:block_size+1]

tensor([480, 239,  52,  78,   9, 401,   4, 223, 265])

In [21]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([480]) the target: 239
when input is tensor([480, 239]) the target: 52
when input is tensor([480, 239,  52]) the target: 78
when input is tensor([480, 239,  52,  78]) the target: 9
when input is tensor([480, 239,  52,  78,   9]) the target: 401
when input is tensor([480, 239,  52,  78,   9, 401]) the target: 4
when input is tensor([480, 239,  52,  78,   9, 401,   4]) the target: 223
when input is tensor([480, 239,  52,  78,   9, 401,   4, 223]) the target: 265


In [22]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 15,  88,  12, 597, 416,  22, 212,   3],
        [ 54,  70,   3,  27, 933, 105,  36,  34],
        [370,   6, 238,  20,   9, 639,  16,  20],
        [ 35, 201,  45, 216, 344,   9, 137,  25]])
targets:
torch.Size([4, 8])
tensor([[ 88,  12, 597, 416,  22, 212,   3, 159],
        [ 70,   3,  27, 933, 105,  36,  34, 186],
        [  6, 238,  20,   9, 639,  16,  20,   8],
        [201,  45, 216, 344,   9, 137,  25,  26]])
----
when input is [15] the target: 88
when input is [15, 88] the target: 12
when input is [15, 88, 12] the target: 597
when input is [15, 88, 12, 597] the target: 416
when input is [15, 88, 12, 597, 416] the target: 22
when input is [15, 88, 12, 597, 416, 22] the target: 212
when input is [15, 88, 12, 597, 416, 22, 212] the target: 3
when input is [15, 88, 12, 597, 416, 22, 212, 3] the target: 159
when input is [54] the target: 70
when input is [54, 70] the target: 3
when input is [54, 70, 3] the target: 27
when input is [54, 70, 3, 27]

In [23]:
print(xb) # our input to the transformer

tensor([[ 15,  88,  12, 597, 416,  22, 212,   3],
        [ 54,  70,   3,  27, 933, 105,  36,  34],
        [370,   6, 238,  20,   9, 639,  16,  20],
        [ 35, 201,  45, 216, 344,   9, 137,  25]])


In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 1000])
tensor(7.6862, grad_fn=<NllLossBackward0>)
 ⁇ v kohus kuidagi ki sellest rääkima rinnare naise suur v tundmusdega sõnad?“ suu perenaisej ema naine sisse re luba aru väljamäe külje mida laulu vaatama K hoo ku väljaPre asi sõi palja aja kogu raba panimise jõudis mõnimiseks tegi: naiseeks vastu täis valmis“ kuu Mse mõni tuli magama mitte sauna laua silmapilk hoid et nõnda teine silmapilgu tahtis las taht all oled rõõm kunagi valmis Rava ku suure sõna selge and taga toalik ningpõlve iga naine har Andres haig jällegi Jaagup rätsep hääl vahel tühja pähe


In [25]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [26]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


7.357106685638428


In [27]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

 ⁇  võimalik Nõnda kahe kõr enam teisedngundda pa m rinna niisama kirjutajasin nukra pitu M kelle tahtor pähe m nüüd järel pidassu aga Kavi teha olnud ehitavär karju tehtud võõra3 sõnu igaK alla küljeMa jumalaus Ometikoolilgu Kassiaru nut tähele kunagi Jõessaare aita ainult silm9 istu koguva täis äkki sööNolik JaMis pühakirja löö teisepere hästi ei Jõessaare hakkab mehe sul elu ki nukra isa teisi ker alguses südame päeva kurja Karla pisut Andresele esimese mätta võina neistliku a tütre üldse KõikAJlikult mõne Nagu pa leidis kaKüllra o võttis püksi poleks jääna seda kivi poisi öelnud võimalik to su nutar näis mõte läinud raske kee Nagu sõ karja ees käesVri täis kelika hoialguEga tagantad kahju öelnud tammiEksJ vägagi seisis meelde kohtuäär üle sulane Need„ hoid mõni jõu heit ja ühe niisugune ühespõlveNo jumala lõhna kunaAnd minemabj poeg nõr häda Sest tea laskis isekku ühes õieti hammas need jõudis enne neist See“ ahju teisepere sauna päeva teind ühes tahtnud olla jällegi hingemumasSa a

## The mathematical trick in self-attention

In [28]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [29]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [30]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [31]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [32]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [33]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [34]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [35]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [36]:
k.var()

tensor(1.0449)

In [37]:
q.var()

tensor(1.0700)

In [38]:
wei.var()

tensor(1.0918)

In [39]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [40]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [41]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [42]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [43]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [44]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [45]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# with open('input.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.213212 M parameters
step 0: train loss 4.6622, val loss 4.6603
step 100: train loss 2.6666, val loss 2.6684
step 200: train loss 2.5070, val loss 2.5203
step 300: train loss 2.4285, val loss 2.4469
step 400: train loss 2.3727, val loss 2.3895
step 500: train loss 2.3162, val loss 2.3342
step 600: train loss 2.2711, val loss 2.2779
step 700: train loss 2.2218, val loss 2.2341
step 800: train loss 2.1843, val loss 2.1882
step 900: train loss 2.1356, val loss 2.1618
step 1000: train loss 2.1152, val loss 2.1331
step 1100: train loss 2.0869, val loss 2.0850
step 1200: train loss 2.0587, val loss 2.0621
step 1300: train loss 2.0294, val loss 2.0457
step 1400: train loss 2.0059, val loss 2.0191
step 1500: train loss 1.9866, val loss 2.0009
step 1600: train loss 1.9673, val loss 1.9927
step 1700: train loss 1.9490, val loss 1.9744
step 1800: train loss 1.9256, val loss 1.9482
step 1900: train loss 1.9124, val loss 1.9314
step 2000: train loss 1.8971, val loss 1.9065
step 2100: train loss 1.